In [ ]:
import pandas as pd
import os
print(os.getcwd())
import matplotlib.pyplot as plt
import seaborn as sns
import re
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 20)


# Data

In [ ]:
data_train=pd.read_parquet("../../../../dataset/full-dataset/raw/train.parquet")
data_test=pd.read_parquet("../../../../dataset/full-dataset/raw/test.parquet")
data = pd.concat([data_train, data_test], axis=0, ignore_index=True)
data.head()


#check exist features

# Columns description

| Column Name          | Description |
|----------------------|-------------|
| `is_multipart`       | Whether the email is a multipart email (i.e., contains multiple parts like text and attachments). |
| `From`               | Full `From` field in raw format, often includes display name and email address. |
| `From_name`          | Display name of the sender, parsed from the `From` field. |
| `From_email`         | Email address of the sender. |
| `From_email_domain`  | Domain portion of the sender’s email address (after `@`). |
| `To`                 | Full `To` field in raw format, possibly containing multiple addresses. |
| `To_name`            | Display name(s) in the `To` field, if available. |
| `To_email`           | Parsed email address from the `To` field. |
| `To_email_domain`    | Domain portion of the recipient’s email address. |
| `Subject`            | Subject line of the email. |
| `Content_types`      | List of MIME content types present in the email (e.g., `text/plain`, `text/html`, `application/pdf`). |
| `text_plain`         | Plain text content extracted from the email body. |
| `text_clean`         | Preprocessed plain text (cleaned version of `text_plain`), typically lowercased and stripped of formatting. |
| `text_html`          | HTML content extracted from the email body, if present. |
| `text_preprocessed`  | Tokenized and normalized version of the email body text for modeling (could be used for NLP). |
| `text_hyperlinks`    | List of hyperlinks or `mailto:` links found in the body content. |
| `attachment_types`   | List of MIME types of any attachments in the email (e.g., `[application/pdf]`). |
| `target_1`           | Binary label for classification (e.g., `malicious` vs `benign`). |
| `target_2`           | More specific label for intent (e.g., `phishing`, `legitimate`). |
| `target_3`           | Granular label describing type/source of the email (e.g., `self_phishing`, `third_party`, `legitimate_email`). |


---

# is_multipart

In [ ]:
counts = data['is_multipart'].value_counts(dropna=False).sort_index()

ax = counts.plot(kind='bar', figsize=(6, 4), color=['steelblue', 'orange'])
plt.title("is_multipart distribution")
plt.xlabel("is_multipart")
plt.ylabel("Count")
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=0)

In [ ]:
target_dist = data.groupby('is_multipart')['target_1'].value_counts(normalize=False).unstack().fillna(0)

target_percent = target_dist.div(target_dist.sum(axis=1), axis=0).round(3) * 100 

ax = target_dist.plot(kind='bar', stacked=True, figsize=(8, 5), colormap='Set2')
plt.title("target_1 Distribution by is_multipart")
plt.xlabel("is_multipart")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.legend(title='target_1')

for i in range(target_dist.shape[0]):
    total = target_dist.iloc[i].sum()
    cum = 0
    for j, val in enumerate(target_dist.iloc[i]):
        y = cum + val / 2
        pct = target_percent.iloc[i, j]
        ax.text(i, y, f"{pct:.1f}%", ha='center', va='center', fontsize=9)
        cum += val

plt.tight_layout()
plt.show()


In [ ]:
def plot_stacked_target_by_is_multipart(target_col):
    target_dist = data.groupby('is_multipart')[target_col].value_counts(normalize=False).unstack().fillna(0)
    target_percent = target_dist.div(target_dist.sum(axis=1), axis=0).round(3) * 100

    fig, ax = plt.subplots(figsize=(10, 6))
    target_dist.plot(kind='bar', stacked=True, ax=ax, colormap='tab20')

    ax.set_title(f"{target_col} Distribution by is_multipart")
    ax.set_xlabel("is_multipart")
    ax.set_ylabel("Count")
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    ax.set_xticklabels(target_dist.index, rotation=0)

    for i in range(target_dist.shape[0]):
        total = target_dist.iloc[i].sum()
        cum = 0
        for j, val in enumerate(target_dist.iloc[i]):
            y = cum + val / 2
            pct = target_percent.iloc[i, j]
            if pct > 1:
                ax.text(i, y, f"{pct:.1f}%", ha='center', va='center', fontsize=9)
            cum += val

    ax.legend(title=target_col, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()
    plt.show()

plot_stacked_target_by_is_multipart("target_2")



# From and To

In [ ]:
email_cols = [
    'From', 'From_name', 'From_email', 'From_email_domain',
    'To', 'To_name', 'To_email', 'To_email_domain'
]

email_col_summary = []

for col in email_cols:
    nunique = data[col].nunique(dropna=False)
    missing = data[col].isnull().sum()
    email_col_summary.append({
        "Column": col,
        "Unique Values": nunique,
        "Missing Values": missing
    })

email_summary_df = pd.DataFrame(email_col_summary)
email_summary_df

In [ ]:
columns = [
    'From', 'From_name', 'From_email', 'From_email_domain',
    'To', 'To_name', 'To_email', 'To_email_domain'
]

for col in columns:
    print(f"Top 10 values in column: {col}")
    print(data[col].value_counts(dropna=False).head(10).to_frame('Count'))


From_email_domain vs target_1

In [ ]:
top_domains = data['From_email_domain'].value_counts().nlargest(20).index

df_top = data[data['From_email_domain'].isin(top_domains)]

count_table = pd.crosstab(df_top['From_email_domain'], df_top['target_1'])

count_table.plot(kind='bar', stacked=True, figsize=(12,6))
plt.title('Top 20 From_email_domain vs target_1 Counts')
plt.xlabel('From_email_domain')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.legend(title='target_1')
plt.tight_layout()
plt.show()

From_email_domain vs target_2

In [ ]:
top_domains = data['From_email_domain'].value_counts().nlargest(20).index

df_top = data[data['From_email_domain'].isin(top_domains)]

count_table = pd.crosstab(df_top['From_email_domain'], df_top['target_2'])

count_table.plot(kind='bar', stacked=True, figsize=(12,6))
plt.title('Top 20 From_email_domain vs target_2 Counts')
plt.xlabel('From_email_domain')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.legend(title='target_2')
plt.tight_layout()
plt.show()


To_email_domain vs target_1

In [ ]:
top_domains = data['To_email_domain'].value_counts().nlargest(20).index

df_top = data[data['To_email_domain'].isin(top_domains)]

count_table = pd.crosstab(df_top['To_email_domain'], df_top['target_1'])

count_table.plot(kind='bar', stacked=True, figsize=(12,6))
plt.title('Top 20 To_email_domain vs target_1 Counts')
plt.xlabel('To_email_domain')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.legend(title='target_1')
plt.tight_layout()
plt.show()

To_email_domain vs target_2

In [ ]:
top_domains = data['To_email_domain'].value_counts().nlargest(20).index

df_top = data[data['To_email_domain'].isin(top_domains)]

count_table = pd.crosstab(df_top['To_email_domain'], df_top['target_2'])

count_table.plot(kind='bar', stacked=True, figsize=(12,6))
plt.title('Top 20 To_email_domain vs target_2 Counts')
plt.xlabel('To_email_domain')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.legend(title='target_2')
plt.tight_layout()
plt.show()

# Subject

In [ ]:
subject_missing = data['Subject'].isnull().sum()
subject_unique = data['Subject'].nunique(dropna=False)
top_subjects = data['Subject'].value_counts(dropna=False).head(15)
top_subjects

In [ ]:
top_subjects = data['Subject'].value_counts().head(15).index.tolist()

df_top_subjects = data[data['Subject'].isin(top_subjects)]

subject_target_crosstab = pd.crosstab(df_top_subjects['Subject'], df_top_subjects['target_1'])

subject_target_crosstab.plot(kind='barh', stacked=True, figsize=(10, 7), colormap='Set2')
plt.title("Top 15 Subjects vs target_1")
plt.xlabel("Email Count")
plt.ylabel("Subject")
plt.tight_layout()
plt.show()

# Received

In [ ]:
data['Received_str'] = data['Received'].apply(lambda x: '\n'.join(x) if isinstance(x, list) else str(x))
data[['Received_str']].head()


# Authentication-Results (spf_result,dkim_result,dmarc_result)

In [ ]:
auth_null_count = data['Authentication-Results'].isnull().sum()
auth_null_ratio = round(auth_null_count / len(data) * 100, 2)

pd.DataFrame({
    "Metric": ["Missing Values", "Missing Ratio (%)"],
    "Value": [auth_null_count, f"{auth_null_ratio}%"]
})

In [ ]:
def extract_auth_result(field, method):
    pattern = rf'{method}\s*=\s*(\w+)'
    if isinstance(field, str):
        match = re.search(pattern, field, re.IGNORECASE)
        return match.group(1).lower() if match else 'none'
    return 'none'

data['spf_result'] = data['Authentication-Results'].apply(lambda x: extract_auth_result(x, 'spf'))
data['dkim_result'] = data['Authentication-Results'].apply(lambda x: extract_auth_result(x, 'dkim'))
data['dmarc_result'] = data['Authentication-Results'].apply(lambda x: extract_auth_result(x, 'dmarc'))


vs target_1

In [ ]:
for col in ['spf_result', 'dkim_result', 'dmarc_result']:
    plt.figure(figsize=(6,4))
    sns.countplot(x=col, hue='target_1', data=data, order=data[col].value_counts().index)
    plt.title(f"{col} vs target_1")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.legend(title="target_1")
    plt.tight_layout()
    plt.show()


vs target_2

In [ ]:
for col in ['spf_result', 'dkim_result', 'dmarc_result']:
    plt.figure(figsize=(6,4))
    sns.countplot(x=col, hue='target_2', data=data, order=data[col].value_counts().index)
    plt.title(f"{col} vs target_2")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.legend(title="target_2")
    plt.tight_layout()
    plt.show()

# Return-Path

In [ ]:
print(data['Return-Path'])
data['return_path_missing'] = data['Return-Path'].isnull()

In [ ]:
return_path_null_count = data['Return-Path'].isnull().sum()
return_path_null_ratio = round(return_path_null_count / len(data) * 100, 2)

pd.DataFrame({
    "Metric": ["Missing Return-Path Count", "Missing Ratio (%)"],
    "Value": [return_path_null_count, f"{return_path_null_ratio}%"]
})


In [ ]:
return_path_counts = data['Return-Path'].value_counts(dropna=False)

return_path_counts.head(10)


## Return_path match From_email?

In [ ]:
def extract_domain(email):
    if isinstance(email, str) and '@' in email:
        return email.strip().split('@')[-1].lower()
    return None

data['return_path_domain'] = data['Return-Path'].apply(extract_domain)
data['return_path_match'] = data['return_path_domain'] == data['From_email_domain']
data['return_path_match'].value_counts(dropna=False)

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='return_path_match', hue='target_1', data=data)
plt.title("Return-Path Domain Matches From_email Domain?")
plt.xlabel("Match")
plt.ylabel("Email Count")
plt.tight_layout()
plt.show()


# Content-Language

In [ ]:
data['Content-Language'].value_counts(dropna=False)


In [ ]:
top_langs = data['Content-Language'].value_counts().head(8).index

filtered = data[data['Content-Language'].isin(top_langs)]

plt.figure(figsize=(10, 5))
sns.countplot(x='Content-Language', hue='target_1', data=filtered)
plt.title("Top 8 Content-Language vs target_1")
plt.xlabel("Content-Language")
plt.ylabel("Count")
plt.legend(title='target_1')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
top_langs = data['Content-Language'].value_counts().head(8).index

filtered = data[data['Content-Language'].isin(top_langs)]

plt.figure(figsize=(10, 5))
sns.countplot(x='Content-Language', hue='target_2', data=filtered)
plt.title("Top 8 Content-Language vs target_2")
plt.xlabel("Content-Language")
plt.ylabel("Count")
plt.legend(title='target_2')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Reply

In [ ]:
data['Reply-To_domain'].value_counts(dropna=False)

In [ ]:
top_domains = data['Reply-To_domain'].value_counts().dropna().head(10).index

filtered = data[data['Reply-To_domain'].isin(top_domains)]

plt.figure(figsize=(12, 6))
sns.countplot(data=filtered, x='Reply-To_domain', hue='target_1')
plt.title("Top Reply-To Domains vs target_1")
plt.xlabel("Reply-To Domain")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')
plt.legend(title="target_1")
plt.tight_layout()
plt.show()


In [ ]:
top_domains = data['Reply-To_domain'].value_counts().dropna().head(10).index

filtered = data[data['Reply-To_domain'].isin(top_domains)]

plt.figure(figsize=(12, 6))
sns.countplot(data=filtered, x='Reply-To_domain', hue='target_2')
plt.title("Top Reply-To Domains vs target_2")
plt.xlabel("Reply-To Domain")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')
plt.legend(title="target_2")
plt.tight_layout()
plt.show()


# Content-types

In [ ]:
print(data['Content_types'])

In [ ]:
exploded = data.explode('Content_types')

exploded_non_null = exploded.dropna(subset=['Content_types'])

type_counts = exploded_non_null['Content_types'].value_counts()
type_counts.head(10)


In [ ]:
null_count = data['Content_types'].isnull().sum()
null_ratio = round(null_count / len(data) * 100, 2)

print(f"Missing count: {null_count}")
print(f"Missing ratio: {null_ratio}%")


In [ ]:
data['has_pdf'] = data['Content_types'].apply(
    lambda x: 'application/pdf' in x if isinstance(x, list) else False
)

plt.figure(figsize=(6, 4))
sns.countplot(x='has_pdf', hue='target_1', data=data)
plt.title("Presence of application/pdf vs target_1")
plt.xlabel("Contains application/pdf?")
plt.ylabel("Count")
plt.legend(title="target_1")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='has_pdf', hue='target_2', data=data)
plt.title("Presence of application/pdf vs target_2")
plt.xlabel("Contains application/pdf?")
plt.ylabel("Count")
plt.legend(title="target_2")
plt.tight_layout()
plt.show()

# attachment_types

In [ ]:
print(data['attachment_types'])

In [ ]:
exploded_attach = data.explode('attachment_types')

exploded_attach = exploded_attach[exploded_attach['attachment_types'].notna() & (exploded_attach['attachment_types'] != '')]

attachment_counts = exploded_attach['attachment_types'].value_counts()
attachment_counts.head(10)


---

In [ ]:
data = data[data['target_3'] != 'self_phishing']

In [ ]:
counts = data['target_1'].value_counts(dropna=False).sort_index()

ax = counts.plot(kind='bar', figsize=(6, 4), color=['#1f77b4','#ff7f0e'])
plt.title("target distribution")
plt.xlabel("target_1")
plt.ylabel("Count")
plt.xticks(rotation=0)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

counts = (
    data
    .groupby(['Content-Language', 'target_1'])
    .size()
    .reset_index(name='n')
)

wide = counts.pivot(
    index='Content-Language',
    columns='target_1',
    values='n'
).fillna(0)

wide['total'] = wide.sum(axis=1)
wide = wide.sort_values(by='malicious', ascending=False)

N = 10
topN = wide.head(N).drop(columns='total')

plt.figure(figsize=(6, 4))
topN.plot(
    kind='bar',
    color=['#1f77b4','#ff7f0e'],
    width=0.8,
    ax=plt.gca()
)

plt.title(f"Top {N} Content-Language vs target_1", fontsize=14)
plt.xlabel("Content-Language", fontsize=12)
plt.ylabel("Email Count", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='target_1', fontsize=10, title_fontsize=12, loc='upper right')
plt.tight_layout()
plt.show()


In [ ]:
for col in ['spf_result', 'dkim_result']:
    plt.figure(figsize=(6,4))
    sns.countplot(
        x=col,
        hue='target_1',
        data=data,
        order=data[col].value_counts().index,
        hue_order=['benign','malicious'],
        palette={'benign':'#ff7f0e',
                 'malicious':'#1f77b4'}
    )
    plt.title(f"{col} vs target_1")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.legend(title='target_1', loc='upper right')
    plt.tight_layout()
    plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
topN.plot(
    kind='bar',
    stacked=False,
    color=['#1f77b4', '#ff7f0e'],
    width=0.8,
    ax=ax
)

ax.set_title("Top 20 From_email_domain: benign vs malicious", pad=12)
ax.set_xlabel("From_email_domain")
ax.set_ylabel("Count")

# Rotate and align
ax.tick_params(axis='x', labelrotation=45)
ax.set_xticklabels(ax.get_xticklabels(), ha='right')

ax.legend(
    title='target_1',
    loc='upper left',
    bbox_to_anchor=(1.02, 1),
    borderaxespad=0
)

plt.subplots_adjust(bottom=0.25, right=0.8)
plt.tight_layout()
plt.show()
